<!-- vscode-jupyter-toc -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
<a id='toc0_'></a>**Содержание**    
- [Синхронизация потоков](#toc1_)    
    - [Это и называется "состояние гонки"](#toc1_1_1_)    
  - [Состояние гонки](#toc1_2_)    
  - [Критическая секция](#toc1_3_)    
  - [Блокировка](#toc1_4_)    
  - [Свойство взаимного исключения](#toc1_5_)    
    - [Разные блокировки существуют независимо друг от друга](#toc1_5_1_)    
  - [Свойство живости](#toc1_6_)    
  - [Взаимное исключение с использованием RW регистров](#toc1_7_)    
  - [Атомарный Read/Write регистр](#toc1_8_)    
  - [Взаимное исключение для 2-х потоков](#toc1_9_)    
  - [Альтернация](#toc1_10_)    
    - [Свойство взаимного исключения](#toc1_10_1_)    
    - [Свойство живости](#toc1_10_2_)    
  - [Флаги намерения](#toc1_11_)    
  - [Алгоритм Петерсона для 2-х потоков (busy waiting + strict alternation)](#toc1_12_)    
  - [N потоков](#toc1_13_)    
- [Обобщение алгоритма Петерсона для N-х потоков](#toc2_)    
  - [Через вспомогательную структуру](#toc2_1_)    
  - [Через вспомогательную структуру](#toc2_2_)    
  - [Алгоритм O(N) по памяти](#toc2_3_)    
  - [Честность (starvation freedom)](#toc2_4_)    
  - [Супер честность (k-bounded waiting)](#toc2_5_)    
- [Атомарный Read/Modify/Write регистр](#toc3_)    
  - [Взаимное исключение с использованием RWM регистра](#toc3_1_)    
  - [Ticket lock](#toc3_2_)    
  - [И снова о прерываниях](#toc3_3_)    
  - [Deadlock](#toc3_4_)    
  - [Мораль.](#toc3_5_)    
  - [Однопроцессорные системы](#toc3_6_)    
- [Разделение на читателей и писателей](#toc4_)    
  - [Стратегии ожидания (spinlock vs mutex)](#toc4_1_)    
  - [Альтернативы активному ожиданию (mutex)](#toc4_2_)    
  - [Без блокировок](#toc4_3_)    
  - [spinlock -> mutex](#toc4_4_)    
- [Задача Producer-а и Consumer-а](#toc5_)    
  - [Переменная состояния](#toc5_1_)    
  - [Producer](#toc5_2_)    
  - [Consumer](#toc5_3_)    
  - [Зачем нам lock?](#toc5_4_)    
  - [Зачем нам цикл?](#toc5_5_)    
    - [Spurious wakeups (ложные пробуждения) - ситуация, когда wait возвращает управление, даже если никто несигналил](#toc5_5_1_)    
- [Deadlock-и и средства борьбы с ними](#toc6_)    
  - [Deadlock](#toc6_1_)    
  - [Предотвращение deadlock-ов](#toc6_2_)    
    - [Пример](#toc6_2_1_)    
  - [Предотвращение deadlock-ов](#toc6_3_)    
- [Wait-Die](#toc7_)    
  - [Изменим интерфейс блокировки](#toc7_1_)    
  - [Как использовать Wait-Die подход?](#toc7_2_)    
  - ["Контекст"](#toc7_3_)    
  - [Магия timestamp](#toc7_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- /vscode-jupyter-toc -->

# <a id='toc1_'></a>[Синхронизация потоков](#toc0_)

    // пусть есть такой код на C
    int counter;
    int add_one(void) { return ++counter; }     
    
    // его возможная реализация на ASM
        .data
    counter:
        .int 0
        .text
    add_one:
        movq counter, %rax
        inc %rax
        movq rax, counter
        retq

Тогда если есть 2 потока:

        Поток1              Поток2
        ...
        movq counter, %rax
        inc %rax
        movq rax, counter   ...
                            movq counter, %rax
                            inc %rax
                            movq rax, counter
                            ...

... counter=2. Все ОК.

Или

            Поток1
        Поток1              Поток2
        ...
        movq counter, %rax  ...
        ...                 movq counter, %rax
        inc %rax            ...
        movq rax, counter   ...
        ...                 inc %rax
                            movq rax, counter
                            ...
... counter=1 (2-й поток увеличит на 1 значение counter восстановленное в RAX из стека при переключени контекста)

### <a id='toc1_1_1_'></a>[Это и называется "состояние гонки"](#toc0_)

## <a id='toc1_2_'></a>[Состояние гонки](#toc0_)

Состояние гонки - результат зависит от порядка выполнения инструкций
    
    порядок зависит от слишком многих факторов;
    решения планировщика, влияние других потоков (обычно - асинхронны), прерывания (полностью асихронны по отн.к коду)...
    могут быть трудно воспроизводимы - не поддаются тестированию.

## <a id='toc1_3_'></a>[Критическая секция](#toc0_)

Критическая секция
    
    участок кода, обращающийся к разделяемым несколькими потоками данным;
    если не более, чем один поток может одновременно находиться в критической секции, то не будет состояния гонки.
*) в примере: counter - разделяемая между потоками переменная

## <a id='toc1_4_'></a>[Блокировка](#toc0_)

Блокировка (lock) - некоторый объект и пара методов для работы с ним. Еще называется "примитив синхронизации".

    lock - метод захвата блокировки (вызывается перед входом в критическую секцию);
    unlock - метод освобождения блокировки (после крит.секции).

## <a id='toc1_5_'></a>[Свойство взаимного исключения](#toc0_)

Блокировка должна давать гарантию взaимного исключения (mutual exclusion). Эта гарантия включает следующее:

    потоки всегда вызывают lock и unlock парами (сначала lock, а потом unlock);
    не более одного потока может одновременно находиться между lock-ом и unlock-ом.

Пример:

    // пусть есть такой код на C
    struct lock lock;
    int counter;

    int add_one(void) {
        int res;
        lock(&lock);        // т.к. из этого вызова (если несколько вызывающих) возврат будет дан только одному
        res = ++counter;    // то тут будет только 1 поток 
        unlock(&lock);      // разблокировка - следующий лок у другого потока получит возможность возврата
        return res; 
    }    

*) в данном примере имя lock используется сразу для 3-х разных сущностей: структуры, переменной, и функции. Что конечно не педагогично совсем.

    struct lock l;
    int counter;

    int add_one(void)       // в таком варианте гонка не устранится, т.к. 
    {                       // разделяемые данные меняются (через неразделяемую локальную переменную)
        int res;            // без блокировки

        lock(&l);
        res = counter;
        unlock(&l);

        ++res;

        lock(&l);
        counter = res;
        unlock(&l);

        return res;
    }

### <a id='toc1_5_1_'></a>[Разные блокировки существуют независимо друг от друга](#toc0_)

    struct lock lock0;
    int counter0;

    int add_one(void) {
        int res;
        lock(&lock0);       // не мешает делать другие блокировки в других потоках
        res = ++counter0;    
        unlock(&lock0);  
        return res; 
    }    

    struct lock lock1;
    int counter1;

    int add_one(void) {
        int res;
        lock(&lock1);       // защищена другим (экземпляром) объекта блокировки
        res = ++counter1;    
        unlock(&lock1);  
        return res; 
    }   

## <a id='toc1_6_'></a>[Свойство живости](#toc0_)

    struct lock { };    // гарантию взаимного исключения в блокировке можно сделать так, но этого недостаточно
                        
    void lock(struct lock *unused) {
        (void) unused;
        while(1);       // т.к. любой попытавшийся захватить блокировку зависнет
    }

    void unlock(struct lock *unused) {
        (void) unused;
    }

Свойство живости (deadlock freedom)

    если один из потоков вызвал lock, то какой-то из потоков, вызвавших lock, захватит блокировку;
    поток не ждет в lock, если он единственный пытается захватить блокировку;
    если поток ждет, значит другому потоку повезло захватить блокировку.

На что нельзя полагаться? Скорость работы потоков:

    мы не знаем, сколько времени потребуется потоку, чтобы выполнить какой-то код;
    мы не можем полагать, что какой-то поток быстрее.

На что можно полагаться? Потоки работают корректно:

    поток не находится между lock и unlock бесконечно;
    поток не "падает" (seg_fault например), находясь между lock и unlock;
    и так далее...

Как же тогда реализуется блокировка


## <a id='toc1_7_'></a>[Взаимное исключение с использованием RW регистров](#toc0_)

## <a id='toc1_8_'></a>[Атомарный Read/Write регистр](#toc0_)

Атомарный RW регистр - ячейка памяти и пара операций
    
    write - "атомарно" записывает значение в регистр;
    read - "атомарно" читает последнее записанное значение;
    атомарность тут сводится к тому, что все операции (read/write) упорядочены, никакая не может начаться до завершения другой.

\*) атомарный регистр - это абстракция, как она реализуется на реальном оборудования для рассмотрения алгоритмов, которые ей пользуются не важно  
\*) атомарный регистр может буть реализован как обычная ячейка памяти, но для выполнения операций над ней используются специальные инструкции, никакой проблемы тут нет

мы можем использовать более чем один атомарный регистр. В этом случае мы преполагаем, что так же, как и с одним атомарным регистром, операции над всеми регистрами упорядочены и этот порядок согласуется с порядком инструкций в программе.

    atomic_int a, b, c, d;

    void thread0(void) {
        atomic_store(&c, 1);
        atomic_store(&a, atomic_load(&d));
    }
    void thread1(void) {
        atomic_store(&d, 1);
        atomic_store(&b, atomic_load(&c));
    }

Пусть один поток выполняет функцию thread0, а другой поток выполняет функцию thread1. Начальные значения атомарных регистров a, b, c и d равны 0. Какие значения могут хранится в атомарных регистрах a, b, c и d по завершению обеих функций?

-> с, d точно равны 1, a = 0 или 1, b = 0 или 1, кроме варианта a = b = 0.

## <a id='toc1_9_'></a>[Взаимное исключение для 2-х потоков](#toc0_)

Есть всего два потока (реализация для 2-х масштабируется на любое число потоков)
    
    потоки имеют идентификаторы 0 и 1;
    внутри потока мы можем узнать его идентификатор (пусть за это отвечает функция threadId).

## <a id='toc1_10_'></a>[Альтернация](#toc0_)

Возможная (некорректная) реализация блокировки (должно скомпилироваться с -С11):

    #include <stdatomic.h>  // atomic_store, atomic_load - атомарные запись/чтение в дин.память + atomic_int

    struct lock { atomic_int last; };
    
    void lock_init(struct lock *lock) {
        atomic_store(&lock->last, 0);                       // путает лектор по максимуму - 0 тут это и есть наш pid, у второго 1
    }

    void lock(struct lock *lock) {
         while (atomic_load(&lock->last) == threadId());    // while(true) для pid != 0; 
    }

    void unlock(struct lock *lock) {
        atomic_store(&lock->last, threadId());              // тут мы восстановим 0, т.е. разрешим другим заходить
    }


### <a id='toc1_10_1_'></a>[Свойство взаимного исключения](#toc0_)

Для приведенного алгоритма взаимное исключение гарантируется
    
    lock может вернуть управление только потоку с идентификатором, не равным lock−>last;
    только поток с threadId() != lock−>last можети зменить значение lock−>last.

### <a id='toc1_10_2_'></a>[Свойство живости](#toc0_)

Пусть поток 1 вообще никогда не пытается захватить лок
    
    если поток 0 вызовет lock, то он зависнет навсегда;
    т. е. свойство живости не выполняется (но не совсем).

*) Strict Alternation is not deadlock-freedom  
Если первый поток повиснет в функции блокировки, причем насовсем, если второй поток не попытается захватить блокировку и не поменяет атомарный регистр.

## <a id='toc1_11_'></a>[Флаги намерения](#toc0_)

Это другая (более настоящая) реализации блокировки

    #include <stdatomic.h> 

    struct lock { atomic_int flag[2]; };
    
    void lock_init(struct lock *lock) {
        atomic_store(&lock->flag[0], 0);        // нет намерения блокировки                   
        atomic_store(&lock->flag[1], 0);                       
    }

    void lock(struct lock *lock) {
        const int me = threadId();              
        const int other = 1 - me;
        atomic_store(&lock->flag[me], 1);       // есть намерение блокировки

        while (atomic_load(&lock->flag[other]);  
    }

    void unlock(struct lock *lock) {
        const int me = threadId();
        atomic_store(&lock->flag[me], 0);
    }

*) опять херь - оба одновременно добрались до while, то оба установили намерение - а значит оба зависнут в while(true). Гарантируется только взаимное исключение, но не живость

... итак, 2 неправильных алгоритма, как же надо?

## <a id='toc1_12_'></a>[Алгоритм Петерсона для 2-х потоков (busy waiting + strict alternation)](#toc0_)

Объединение двух примеров выше:

    #include <stdatomic.h> 

    struct lock { 
        atomic_int last;
        atomic_int flag[2]; 
    };
    
    void lock(struct lock *lock) {
        const int me = threadId();              
        const int other = 1 - me;

        atomic_store(&lock->flag[me], 1);       // есть намерение блокировки
        atomic_store(&lock->last, me)           // последний - я (кто первый записал, тот и проскочит while(true))
        // если поменять порядок, то взаимное исключение не гарантируется...

        while ( atomic_load(&lock->flag[other] &&     // если у других есть намерения
                atomic_load(&lock->last == me ) ;     // и last мой - то ждать (значит другой первым поставил свой last)
    }

    void unlock(struct lock *lock) {
        const int me = threadId();
        atomic_store(&lock->flag[me], 0);       // нет намерения блокировки
    }

Взаимное исключение

    Доказательство от противного - пусть два потока одновременно находятся в критической секции
        оба потока записывали значение в атомарный регистр last;
        один из них должен был быть первым, а другой последним;
        для определенности пусть последним был поток 1.
    Итак нам известно следующее:
        lock−>last == 1 - последним туда записал поток 1;
        lock−>flag[0] = 1 и lock−>flag[1] == 1.
    Как в таких условиях поток 1 мог пройти мимо цикла в lock и войти в критическую секцию?
        очевидно, никак.

Живость
    
    Пусть поток 0 пытается войти в критическую секцию, возможны две ситуации:
        при проверке условия цикла lock−>flag[1] == 0;
        при проверке условия цикла lock−>flag[1] == 1.
    В первом случае (lock−>flag[1] == 0)
        поток 1 даже не пытался захватить блокировку;
        условие цикла, очевидно, ложно, и поток 0 входит вкритическую секцию
    Во втором случае (lock−>flag[1] == 1)
        оба потока изъявили намерение войти в критическую секцию;
        нужно показать, что хотя бы один из них рано или поздно войдет в критическую секцию (или уже там).
    Оба потока после записи в lock−>flag[x] должны в какой-то момент записать в lock−>last
        не трудно увидеть, что если lock−>flag[0] == 1 и lock−>flag[1] == 1, 
        то тот из них, кто сделал это первым, войдет вкритическую секцию.


## <a id='toc1_13_'></a>[N потоков](#toc0_)

Реализовав взаимное исключение для 2-х потоков, мы можем реализовать вазимное исключение для любого числа потоков

    организуем турнир для N потоков;
    потоки конкурируют друг с другом на "выбывание" (по парам).
        в каждой паре проходит один второй ждет (до тех пор когда его соперник отпустит блокировку)
            а он отпустит, только когда получит ее по итогам всего "турнира"

# <a id='toc2_'></a>[Обобщение алгоритма Петерсона для N-х потоков](#toc0_)

## <a id='toc2_1_'></a>[Через вспомогательную структуру](#toc0_)

    struct lock_one { 
        atomic_int last;
        atomic_int flag[N];                                 // N флагов
    };

    int flags_clear(const struct lock_one *lock) {
        const int me = threadId();              
        for (int i=0; i!=N; ++i) 
            if (i != me && atomic_load(&lock->flag[i])
                return 0;                                  // если кто-то, кроме нас != 0
        return 1;                                          // если все, кроме на == 0
    }

    void lock_one(struct lock *lock) {
        const int me = threadId();              
        const int other = 1 - me;

        atomic_store(&lock->flag[me], 1);       
        atomic_store(&lock->last, me)           
        
        while ( !flags_clear(lock) &&                       // если у кого-то есть намерение блокировки
            atomic_load(&lock->last == me ) ;               
    }

    void unlock_one(struct lock *lock) {
        const int me = threadId();
        atomic_store(&lock->flag[me], 0);       
    }

Не гарантирует исключения доступа. Т.к. блокируется только 1 поток.  
Алгоритм берется за основу для выявления 1 проигравшего, для которого блокируется поток.

## <a id='toc2_2_'></a>[Через вспомогательную структуру](#toc0_)

    struct lock { 
        struct lock_one lock[N - 1];                // N - 1 вспомогательная конструкция
    };

    void lock(struct lock *lock) {
        for (int i=0; i!=N-1; ++i)           
            lock_one(&lock->lock[i])                // чтобы пройти блокировку нужно N-1 раз не оказаться последним
    }

    void unlock(struct lock *lock) {
        for (int i=N-2; i >= 0; --i)           
            unlock_one(&lock->lock[i])
    }

Так будет работать. Но у нас N*(N-1) флагов в N-1 локах. Требуется O(N**2) памяти (регистров).   
Учитывая дублирование значений флагов для каждого потока i в каждой вспомогательной структуре j<i можно оптимизировать все это.
Т.е. до уровня, до которого дошел поток в конкуренции за блокировку его флаги в структурах будут 1, выше этого уровня 0. Поэтому достаточно только 1 числа для кодирования этого префикса.

## <a id='toc2_3_'></a>[Алгоритм O(N) по памяти](#toc0_)

    struct lock { 
        atomic_int level[N];                                // префикс из предыдущего вариант (уровень, до которого дошел i поток)
        atomic_int last[N-1];                               // N-1 потоков, которые могли бы обогнать данный
    };

    int flags_clear(const struct lock_one *lock, int i) {
        const int me = threadId();              
        if (me != atomic_load(&lock->level[i])
            return 0;                                       // если кто-то, кроме нас != 0
        return 1;                                           // если все, кроме на == 0
    }

    void lock_one(struct lock *lock) {
        const int me = threadId();              
      
        for (int i=0; i!=N-1; ++i) 
            atomic_store(&lock->level[me], i + 1);       
            atomic_store(&lock->last[i], me)           
        
            while ( !flags_clear(lock, i) &&                // если у кого-то есть намерение блокировки
                atomic_load(&lock->last[i] == me );               
    }

    void unlock_one(struct lock *lock) {
        const int me = threadId();
        atomic_store(&lock->level[me], 0);       
    }

Доказано, что это оптимальный по памяти алгоритм - нельзя обеспечить взаимное исключение N потоков с исп. менее, чем N-1 атомарного регистра.

## <a id='toc2_4_'></a>[Честность (starvation freedom)](#toc0_)

Не хочется, чтобы потоки голодали!

    если поток захотел захватить блокировку, то когда-нибудь ему это удастся;
    сравните с живостью - среди потоков, пытающихся захватить блокировку, одному это удастся.

*) свойство живости не гарантирует от ситуаций, когда один поток получил блокировку, выполнил работу, и успел опять захватить блокировку. Тогда второй поток теоретически жив, но "голодает", т.е. пропускает свой ход. Либо возможен дед-лок, когда второй поток сможет отработать, только в случае, когда первый полностью перестанет брать себе блокировку.  

Т.е. если мешает получить блокировку не алгоритм, а планировщик ОС, который не дает процессорное время протоку, чтоб он смог блокировку захватить, то честность (алгоритма) не нарушена

## <a id='toc2_5_'></a>[Супер честность (k-bounded waiting)](#toc0_)
    
    k-ограниченное ожидание:
        после того как поток "изъявил" желание захватить блокировку (встал в очередь), не более k потоков могут пролезть вперед него без очереди.
    
\*) побочный эффект такой очереди, она не позволит взять блокировку, даже если ОС дает потоку процессорное время (и он мог бы и сам получить блокировку в честном алгоритме), пока поток не дойдет до начала очереди...

# <a id='toc3_'></a>[Атомарный Read/Modify/Write регистр](#toc0_)

Атомарный RMW регистр позволяет за одну операцию
    
    прочитать значение в регистре;
    преобразовать некоторым образом прочитанное значение;
    записать преобразованное значение назад

    int atomic_rmw(int *reg, int (*f)(int)) {       // вся функция - атомарная
        const int old = *reg;                       // const просто для порядка, не для сути
        const int new = f(old);

        *reg = new;
        return old;
    }

Атомарный Read/Modify/Write регистр 

    atomic_exchange - возвращает старое значение, записывает новое (когда f возвр. константу, как бы атомизация сразу пары load+store);
    atomic_fetch_{add|sub|or|and|xor} - выполняет арифметическое действие над атомарным регистром;
    atomic_compare_exchange - записывает новое значение, если старое значение равно заданному.

Реализация RMW регистра

    Архитектура может поддерживать RMW операции (x86 - одна из них)
        xchg;                                       
        lock add, lock sub, lock or, lock and, lock xor;    // атомарные RMW инструкции, имеющиеся в x86
        lock cmpxchg.

    Архитектура может поддерживать LL/SC (например, ARM):
        LL (load-link, load-linked, load-locked) - загружает значение из памяти;
        SC (store-conditional) - записывает новое значение в ячейку, но только если после LL эту ячейку никто не менял;
        тогда их пара LL/SC работает вместе как одна RMW операция.


Реализация атомарных операций с памятью происходит путём аппаратного блокирования системной шины процессором на время выполнения атомарной операции (которая включает чтение, модификацию и запись). Во время выполнения этих трёх операций выполнение каких-либо других операций на шине невозможно, что снижает производительность других процессров в системе (если они разделяют общую шину), даже если они не имеют никакого отношения к данному спинлоку. 


## <a id='toc3_1_'></a>[Взаимное исключение с использованием RWM регистра](#toc0_)

RW вариант интересен с теоретической т.з. На практике все проще организовать через RWM

    #include <stdatomic.h>  
    #define LOCKED   1
    #define UNLOCKED 0

    struct lock { atomic_int locked; };
    
    void lock(struct lock *lock) {
         while (atomic_exchange(&lock->locked, LOCKED) != UNLOCKED);    // while(true) если не получилось переключить (пока возвращается LOCKED (предыдущее значение))
    }                                                                   // т.е. дальше мы сможем пройти только если вернулось в кач-ве пред.знач. UNLOCKED

    void unlock(struct lock *lock) {
        atomic_store(&lock->last, UNLOCKED);                            // атомарно меняем флаг
    }

И снова про честность. Что если блокировка находится под нагрузкой (high contention)?
    
    т. е. блокировка практически всегда занята;
    некоторый поток может получать CPU только тогда, когда блокировка занята;
    такой поток будет голодать - блокировка не честная.

## <a id='toc3_2_'></a>[Ticket lock](#toc0_)

Более честный вариант RMW блокировки. Косвенная очередь.

    struct lock { 
        atomic_uint ticket;                                         // текущий "талончик"
        atomic_uint next;                                           // следующий, кто получит блокировку
    };
    
    void lock(struct lock *lock) {
        const unsigned ticket = atomic_fetch_add(&lock->ticket, 1); // получаем очередной "талончик"
        while (atomic_load(&lock->next) != ticket);                 // ждем пока наш "талончик" не вызовут
    }                                                               // 

    void unlock(struct lock *lock) {
        atomic_fetch_add(&lock->next, 1);                           // атомарно меняем "талончик" в очереди
    }

## <a id='toc3_3_'></a>[И снова о прерываниях](#toc0_)

Пусть у нас есть устройство, которое получает данные из сети
    
    устройство сигналит процессору - генерирует прерывание;
    процессор вызывает обработчик прерывания - функцию ядра ОС;
    обработчик прерывания должен забрать данные с устройства и положить их в буфер, из которого какой-то поток сможет их забрaть.

Что если к этому буферу могут обращаться из нескольких потоков?

    мы должны защитить буфер блокировкой;
    потоки и обработчики прерываний должны захватывать эту блокировку перед обращением к буферу;
    что если обработчик прерывания устройства прервал поток, который уже захватил эту блокировку?

## <a id='toc3_4_'></a>[Deadlock](#toc0_)

Прерванный поток и обработчик прерывания бесконечно ждут друг друга:

    обработчик прерывания не может захватить блокировку, потому что ее держит прерванный им же самим поток;
    пока обработчик прерывания не завершится, прерванный поток не получит управление и не сможет отпустить блокировку.

\*) обработчик прерывания исполняется в контексте потока, который был прерван, это не другой поток, это тот же самый поток, который держит блокировку. Т. е. другими словами один и тот же поток пытается захватить блокировку два раза - deadlock.  

\*\*) похоже на проблему инверсии приоритетов, если считать, что прерывания - это вроде как тоже блокировка, но от процесса с более высоким приоритетом (на самом же деле приоритеты обработчика и потока в данном случае не важны (если планировщик поддерживает приориеты, то они будут одинаковы), тут речь об особой природе прерываний). Прерывания имеют приоритет на аппаратном уровне, а не уровне ОС.  

        Получается можно было бы выдернуть из такой ситуации другим прерыванием, но при написании обработчиков прерываний принято отключать прерывания (флаг IF=0).
        Теоретически, можно этого не делать, и добавить обработчик прерываний в обработчик прерывании... Он должен получается сохранять контекст текущего обработчика прерываний,
        (контект прерванной функции не должен попортиться), для такого вложенного обработчика не существует блокировки прерванной функции, он ее игнорирует. Но он в любом случае
        кого-то должен дропнуть в этом дедлоке (или функцию или обработчк), определить кто реально накосячил из них в общем случае невозможно, и скорее всего никто, а проблема 
        в логике кода одного или другого. Т.е. принудительно разблокировать можно - но работоспособность и целостность данных гарантировать невозможно таким способом.

\*\*\*) Инверсия приоритетов - это когда более высоко-приоритетный поток вынужден ждать разблокировки от низкоприоритетного, который не имеет возможности разблокироваться из-за своего низкого приоритета (ожидая своей очереди, которая не доходит до него из-за более важного среднеприоритетного потока). Пусть есть потоки минимум 3 уровней приоритета (0(выс.), 1(ср.), 2(низ.)). 2й заблокировал крит.область. Планировщик переключает на 0й (просто потому что тот важнее), тот делает свои дела, если доходит дело до заблокированного ресурса, он становится в ожидание разблокировки. Планировщик переключает по приоритетности на средний поток, которому блокировка не нужна (еслиб была нужна, он бы встал в ожидание и 0-й смог бы продолжить работу, т.е. отдать блокировку - все ок). А так в итоге 0й поток полностью зависит от того, когда соизволит закончить работу средний (1-й). Простое переключение контекста на 0й ничего не даст, он в состоянии ожидания разблокировки. Т.е. 2-й по приоритетности стал самым главным из трех.  
Решается наследованием приоритетов в планировщие задач, когда заблокировавшему ресурсы потоку (2й) приоритет поднимается до наивысшего и приоритетов потоков, ожидающих разблокировки (0й). Он получит свой контекст, разблокирует - все ок. Потом правда надо за ним следить, как только возможно - приоритет вернуть на место.  
 
В CFS Linux приоритетов нет - нет такой проблемы (речь про инверсию приоритетов, а не про дедлоки).    

Из-за такой херни марсоход в 90-х поломался (ОС VxWorks (POSIX, жесткий реал-тайм, 256 уровней приоритетов, вытесняющий планировщик, обработчики прерываний работают вне констекста потоков для скорости), CFS еще не изобрели) - поток сбора погоды (низк.) заблокировал системную шину, к которой обратился "драйвер" шины (выс.), после чего контект переключился на поток модуля связи (ср.) и в нем и остался без возможности переключиться на другую задачу (они ожидали разблокировки). Через сутки, когда разобрались, т.к. ядро позволяло ручное переключение контекста - просто вручную переключили, и все заработало. Потом накатили патч.    
Думается, еще и повезло - если б он завис не на модуле связи, хрен бы его разбудили, и вообще не узнали бы никогда что же произошло с железякой...

Запрет прерываний:

    x86 RFLAGS содержит флаг IF (interrupt flag)
    инструкция cli обнуляет этот флаг - запрет прерываний
    инструкция sti устанавливает флаг обратно


## <a id='toc3_5_'></a>[Мораль.](#toc0_)

Если блокировка защищает данные, к которым обращается обработчик прерывания, то нужно выключать прерывания

    если прерывания отключены, то deadlock между обработчиком прерывания и прерванным потоком не может возникнуть.

## <a id='toc3_6_'></a>[Однопроцессорные системы](#toc0_)

Представим систему с всего одним ядром/процессором
    
    запретив прерывания и не задействуя в своем коде переключение потоков, мы получаем CPU в монопольное пользования;
    все рассмотренные ранее алгоритмы просто не нужны.

# <a id='toc4_'></a>[Разделение на читателей и писателей](#toc0_)

Не все запросы к разделяемым данным одинаковы 

    есть запросы, которые модифицируют данные;
    есть запросы, которые только читают данные.

Пример реализации блокировки, погружающей поток в сон (файлы mutex.h и mutex.c)  
Пример реализации condition varibale (файлы condition.h и condition.c)

Вариант RW лока:

    struct rwlock { 
        atomic_uint ticket;                                         // общий пул "номерков"
        atomic_uint write;                                          // две очереди - для писателей
        atomic_uint read;                                           // - для читателей
    };
    
    void read_lock(struct read_rwlock *lock) {
        const unsigned ticket = atomic_fetch_add(&lock->ticket, 1); // получаем очередной "номерок" и двигаем очередь на 1
        while (atomic_load(&lock->read) != ticket);                 // ждем пока наш "талончик" не вызовут
        atomic_store(&lock->read, ticket + 1);                      // до входа в крит. сразу двигаем очередь читателей (fetch_add тоже ОК)
    }                                                               // следующий читатель поступит также

    void read_unlock(struct rwlock *lock) {
        atomic_fetch_add(&lock->write, 1);                          // только при выходе из крит.обл. продвигаем очередь писателей
    }

    void write_lock(struct read_rwlock *lock) {
        const unsigned ticket = atomic_fetch_add(&lock->ticket, 1); // получаем очередной "талончик"
        while (atomic_load(&lock->write) != ticket);                // ждем пока наш "талончик" не вызовут
    }                                                               // читатели и другие писатели войти не смогут

    void write_unlock(struct rwlock *lock) {
        atomic_fetch_add(&lock->read, 1);                           // атомарно меняем "талончик" в очереди
        atomic_fetch_add(&lock->write, 1);                          // атомарно меняем "талончик" в очереди
    }


## <a id='toc4_1_'></a>[Стратегии ожидания (spinlock vs mutex)](#toc0_)

До сих пор функция lock всегда просто ждала в цикле (кроме разве случая однопроцессорной системы)

    такая стратегия называется активным ожиданием;
    блокировки, использующие активное ожидание, частоназываются spinlock-ами;
    они "крутятся" в цикле.

Активное ожидание хорошо работает если:

    потоки не держат блокировку очень долго;
    блокировка не находится под сильной нагрузкой (много потоков активно пытаются ее перехватить);
    т. е. если активное ожидание длится недолго.

\*) такое ожидание/борьба за блокировку с переключением планировщиком контекста между бесконечными циклами ожидающих потоков - бесполезная трата процессорного времени. Имеет смысл по максимуму отдать контекст потоку, который уже захватил блокировку.

Случаи, когда применение в пространстве пользователя спинлоков даёт ощутимый эффект:

    Внутри участка защищаемого кода находится несколько связанных переменных, время модификации которых может быть в сотни и даже тысячи раз меньше, чем переключение контекста процессором, что является особенно дорогой операцией, особенно на современных системах.
    Блокировка не участков кода, а данных (с каждой структурой данных, которая должна атомарно изменяться как единое целое, связан спинлок, её защищающий)
    Оптимизация кода, когда необходимо уменьшить нагрузку, возникающую за счёт слишком частого переключения контекста


## <a id='toc4_2_'></a>[Альтернативы активному ожиданию (mutex)](#toc0_)

Как можно ожидать не активно?
    
    можно добровольно отдать CPU (переключиться на другой поток) 
        - сам ожидающий поток при нахождении в цикле ожидания spinlock-а должен после скольки-то 
        итераций обратиться к планировщику и попросить не тратить на него время ЦПУ пока 
        что-то не произойдет с блокировкой;
        
    можно пометить поток как неактивный, чтобы планировщик не давал ему время на CPU, пока блокировка не будет отпущена
        - это реализуется отдельным видом блокировщика, который сам определяет, что поток можно 
        таким образом усыпить. Такие блокировщики называются mutex-ами (mutual exclusion).

## <a id='toc4_3_'></a>[Без блокировок](#toc0_)

Существуют также алгоритмы без блокировок, основанные на атомарном детектировании коллизий. Они оптимизированы под оптимистичный случай, при котором вся проверка на коллизию сводится к одной атомарной ассемблерной операции (Compare And Swap, на x86-архитектуре - команда cmpxchg)

## <a id='toc4_4_'></a>[spinlock -> mutex](#toc0_)

Спинлок с автоматическим наращиванием до захвата полноценного мьютекса после истечения какого-то количества оборотов цикла применяется, например, в критических секциях Windows для оптимизации (например, общеизвестная CRITICAL_SECTION, или же FAST_MUTEX в ядре), заключающейся в отсутствии обращений к мьютексу при отсутствии соревнования за ресурс. Такие объекты сочетают лучшие качества спинлоков (минимальная цена захвата) и мьютексов (отсутствие растраты ресурса процессора на опрос). 


# <a id='toc5_'></a>[Задача Producer-а и Consumer-а](#toc0_)

Рассмотрим следующую задачу:

    Producer - поток/потоки, который генерирует данные;
    Consumer - поток/потоки, который потребляет данные;
    что если Producer и Consumer работают с разной скоростью?

## <a id='toc5_1_'></a>[Переменная состояния](#toc0_)
https://en.cppreference.com/w/cpp/thread/condition_variable  

Перемeнная состояния (condition variable) - объект и несколько методов для работы с ним:
    
    wait - метод ожидает, пока кто-нибудь не просигналит;
    notify_one - метод просигналить одному конкретному из ожидающих (тому, кто мониторит соответствующют переменную, "спит на переменной");
    notify_all - метод просигналить всем ожидающим (часто называют broadcast).

Псевдокод объявления:

    struct lock;                                // интерфейс блокировки всегда идет парой к интерфейсу cond.var.
    void lock(struct lock * lock);
    void unlock(struct locl * lock);

    struct condition;                                       // объект состояния
    void wait(struct condition * cv, struct lock * lock);   // требуется, чтобы к вызову wait блокировка была захвачена
                                                            // (передается, т.к. wait ее освободит)
    void notify_one(struct condition * cv);                 // блокировка должна быть захвачена (хотя передавать ее и не нужно)
    void notify_all(struct condition * cv);
    
    

## <a id='toc5_2_'></a>[Producer](#toc0_)

    //две cond.var. (может быть и одна c логикой выбора значения уже внутри, это для наглядности)
    struct condition full;      // сигналит от том что положили значение в буфер
    struct condition empty;     // сигналит от забирании значения из буфера
    struct lock mtx;            // блокировщик мьютекс
    
    int value;                  // то что генерирует продюсер
    bool valid_value;           // флаг наличия значения
    bool done;                  // флаг завершения генерации данных (больше данных не будет)

    // метод потоко-безопасно кладет x в общую переменную value
    void produce(int x) {       // природа генерируемых значений (int *x) не важна
        lock(&mtx);             // забираем блокировку
        while (valid_value)     // ждем на cond.var. empty (КАЗАЛОСЬ БЫ ЗАЧЕМ ТУТ ЦИКЛ)
            wait(&empty, &mtx); // пока не заберут значение из буфера value
        value = x;              // кладем в буфер
        valid_value = true;     // флаг готовности данных
        notify_one(&full);      // сигнал готовности нового значения
        unlock(&mtx);           // разблокируем поток
    }
    // конец генерации данных, финита
    void finish(void) {         // вызывается после produce 
        lock(&mtx);             // блок
        done = true;            // флаг завершения генерации 
        notify_all(&full);      // переменная состояния, которая пробуждает всех клиентов 
                                // full - заберите последний кусок
        unlock(&mtx);           // разблок.
    }    

## <a id='toc5_3_'></a>[Consumer](#toc0_)
    
    // принимает данные по указателю (*x)
    int consumer(int *x) {      
        int ret = 0;            // код нештатного возврата (из блокировки вышли, значения не получили)
        lock(&mtx);             // блок
        while (!valid_value && !done)   
            wait(&full, &mtx);  // поток получатель - в сон до следующего notify_...
        
        if (valid_value) {      // на валидном значении value
            *x = value;         // можно присвоить его общей переменной
            valid_value = false;// флаг, что забрали
            notify_one(&empty); // разбудить следующего в очереди (а это у нас в примере один продюсер)
            ret = 1;            // код штатного возврата
        }
        unlock(&mtx);           // разблок.
        return ret;
    }

\*) wait - ловушка для всех, пришедших сюда потоков, которые он помечает заблокированными для планировщика и пишет в конец очереди ожидающих в объекте condvar. Плюс дергает планировщик для обновления очереди.  
\*\*) notify_one notify_all - помечают первый (все) поток(и) в очереди ожидания как активные для планировщика, теперь им дается время для работы.

## <a id='toc5_4_'></a>[Зачем нам lock?](#toc0_)

    Поток 1                 Поток2
    finish()                consume()

    ...                     /* lock(&mtx) */
    ...                     while (... && !done)
    /* lock(&mtx) */        ...
    done = true;            ...
    notify_all(&full);        ...
    /* unlock(&mtx); */     ...
    ...                         wait(&full, &mtx);
    ...                     ...
    ...                     /* unlock(&mtx); */

Очевидно, функции явно не атомарные, finish может просочиться между проверкой условия и вызовом wait():  
consumer() зависнет в системном вызове wait(), продолжит ждать данные от продюсера, в контексте этого потока done останется false и уже не изменится.

## <a id='toc5_5_'></a>[Зачем нам цикл?](#toc0_)

Пример ситуации, пусть есть два консамера (С0, С1) и один продюсер (Р0). 
    
    -> С0 получает ЦПУ, захватывает блокировку &mtx, встает в ожидание wait(&cv,&mtx), пока значений не сгенерировано.
    -> Затем управление получает Р0, захватывает блокировку &mtx. 
        *) потому что wait() так устроен, внутри wait, инициализированного на переменной состояния и захваченной блокировке,
           захваченная блокировка будет отпущена, поток саспендится пока переменная состояния не true, а потом перед выходом 
           из wait она опять будет захвачена, и так в цикле. Обратный захват гарантируется ОС? И смысл ее отпусткать и захватывать, если 
           блокировка и так у этого потока захвачена ранее?
    -> И тут допустим управление получает С1, и пробует захватить блокировку &mtx, но не может, т.к. она у Р0 (логично), управление у С1 заберут.
    -> Р0 получив управление обратно, генрирует значение и уведомляет ожидающего на переменной &cv С0.
    -> С0 просыпается, чето делает...
    -> Когда управление вернется в Р0 он должен будет отпустить захваченную блокировку &mtx.
        -> Получили соревнование за захват блокировки между C0 (внутри wait()) и С1. Если выиграет C1, то он скушает значение,
            предназначенное для С0.
        -> Когда вернулось управление в С0 он уже не получит значения.

Итого - цикл нужен, чтобы при пробуждении потока вызовом wait() еще раз проверить условие, что разбудили по делу, значение есть, можно работать. Только и всего.    

Но даже если один потребитель и производитель, остается актуальной проблема косячной реализации алгоритма. Даже в этом случае if недостаточно, цикл необходим, т.к.:

### <a id='toc5_5_1_'></a>[Spurious wakeups (ложные пробуждения) - ситуация, когда wait возвращает управление, даже если никто несигналил](#toc0_)
    
    многие реализации переменной состояния подвержены:
        С++;
        Java;
        POSIX Threads...

...they usually happen because, in between the time when the condition variable was signaled and when the waiting thread finally ran, another thread ran and changed the condition.
    
    void wait( std::unique_lock<std::mutex>& lock );	(1) 	(since C++11)

Atomically unlocks lock, blocks the current executing thread, and adds it to the list of threads waiting on *this. The thread will be unblocked when notify_all() or notify_one() is executed. It may also be unblocked spuriously. When unblocked, regardless of the reason, lock is reacquired and wait exits.  
wait() атомарная функция  

/gcc-11.1.0/libstdc++-v3/include/std/condition_variable    

    void
    wait(unique_lock<mutex>& __lock) noexcept;

    template<typename _Predicate>
      void
      wait(unique_lock<mutex>& __lock, _Predicate __p)
      {
	while (!__p())
	  wait(__lock);
      }

*) ну вроде понятно, такое засыпание - это не wait() обеспечивает, а тип блокировки mutex.  От спинлока мьютекс отличается передачей управления планировщику при невозможности захвата мьютекса.

# <a id='toc6_'></a>[Deadlock-и и средства борьбы с ними](#toc0_)

## <a id='toc6_1_'></a>[Deadlock](#toc0_)

Deadlock - ситуация, при которой потоки не могут работать, потому что ждут друг друга:
    
    deadlock потоком исполнения и обработчиком прерывания (решается запретом прерываний);
    поток A ждет, пока поток B что-то сделает (например, отпустит блокировку);
    а поток B ничего не делает, потому что ждет, пока поток A что-то сделает (например, отпустит блокировку).

Пример:

    struct lock a;
    struct lock b;

    void thread0() {
        lock(&a);
        lock(&b);
        ...
        unlock(&a);
        unlock(&b);        
    }

    void thread1() {
        lock(&b);
        lock(&a);
        ...
        unlock(&a);
        unlock(&b);        
    }

    ...             ...
    lock(&a);       ...
    ...             lock(&b);
    lock(&b);       ...             // ждать, пока 1й отпустит &b
    ...             lock(&a);       // ждать, пока 0й отпустит &a
    ...             ...

Wait-for граф - можно изобразить в виде графа ожидания  
Тут он в виде цикла 
    
    th0 (waits)-> b (blocks)-> th1 (waits)-> a (blocks) -> th0 -> ...

Как и с состоянием гонки, deadlock не поддается тестированию

    появление зависит от многих факторов;
    входные данные, решения планировщика, прерывания, производительность оборудования ...

\*) тестирование может показать наличие ошибки, но не доказать ее отсутствие

## <a id='toc6_2_'></a>[Предотвращение deadlock-ов](#toc0_)

Мы хотим избежать появления цикла в wait-for графе
    
    простой случай - все блокировки известны заранее;
    упорядочим все блокировки (например, по адресу, по имени, по любому другому признаку);
    захватываем блокировки только по порядку.

### <a id='toc6_2_1_'></a>[Пример](#toc0_)

    void thread0() {
        lock(&a);
        lock(&b);
        ...
        unlock(&b);
        unlock(&a);        
    }

    void thread1() {
        lock(&c);
        lock(&a);
        ...
        unlock(&c);
        unlock(&a);        
    }

    void thread2() {
        lock(&b);
        lock(&c);
        ...
        unlock(&c);
        unlock(&b);        
    }

Wait-for граф дедлока:

    b -> th1 -> c -> th2 -> a -> th0 -> b -> ...

Отсортируем блокировки a, b и c по алфавиту:

    каждый поток должен захватывать блокировки только согласно порядку;
    например, поток 1 хочет захватить блокировки c и a:
        так как a в алфавте раньше c, то сначала хватаем a,
        потом хватаем c.
    
    void thread1() {
        lock(&a);           // теперь дедлок невозможен
        lock(&c);
        ...

## <a id='toc6_3_'></a>[Предотвращение deadlock-ов](#toc0_)

Сложный случай - все блокировки не известны заранее. Они могут динамически создаваться, передаваться в аргументах под разными именами и в составе различных структур:

    для этого случая придумано много различных вариантов;
    мы рассмотрим подход, который называется Wait-Die.

Рассмотрим следующий пример:

    struct Account {
        struct lock lock;
        int money;
    };

    bool transfer_money(struct Account *from, struct Account *to, unsigned amount)
    {
        lock(&from->lock);
        lock(&to->lock);

        bool success = false;

        if (from->money >= amount) {
            from->money -= amount;
            to->money += amount;
            success = true;
        }

        unlock(&to->lock);
        unlock(&from->lock);

        return success;
    }

При условии, что эта функция может вызываться из нескольких потоков со значениями from и to указывающими на существующие аккаунты и from никогда не равен to, может ли такой код приводить к deadlock-ам?

Да, т.е. объект блокировки передается в аргументы, то возможна ситауция, когда в одном потоке from равен to другого потока (одноврменная пересылка друг другу). Получится цикл как в первом примере. Оба вызова ждут когда противоположный отпустит его второй блок to который равен первому from.

# <a id='toc7_'></a>[Wait-Die](#toc0_)

## <a id='toc7_1_'></a>[Изменим интерфейс блокировки](#toc0_)

    struct wdlock_ctx {                                         // новая структура, обычно называют контекст блокировки
        unsigned long long timestamp;                           // позволяет избегать дедлоков
        struct wdlock *next;                                    // связный список захваченных блокировок
    }

    struct wdlock {
        ...                                                     // должен поддерживать связный список
    }

    /* Grab unique 'timestamp' */
    void wdlock_ctx_init(struct wdlock_ctx *ctx);               // сводится к выбору timestamp

    /* This finc may fail */
    int wdlock(struct wdlock *lock, struct wdlock_ctx *ctx)     // принимает также контекст, возвращает результат 
                                                                // (для отказа в блокировке, если она приводит к дедлоку)

    /* Unlock all of the locks */
    void wdunlock(struct wdlock_ctx *ctx);                      // все блокировки контекста освобождаются

## <a id='toc7_2_'></a>[Как использовать Wait-Die подход?](#toc0_)

    void thread(void) {
        struct wdlock_ctx cxt;                                  // созд. контекст

        wdlock_ctx_init(&ctx);                                  // инициализ. контекст

        while (1) {                                             // цикл захвата необходимых блокировок
            ...

            if (!wdlock(&lock1, &ctx)) {                        // если хоть одна из блокировок не захватилась
                wdunlock(&ctx);                                 // отпустить все блокировки
                continue;                                       // и по-новой
            }
            ..
            if (!wdlock(&lock2, &ctx)) {
                wdunlock(&ctx);
                continue;
            }
            ...                                                 // где-то тут условный break для выхода из цикла
        }
        /*Acquired all reauired locks saccesfully*/         
        ...                                                     // полезная работа потока
        wdunlock(&ctx);                                         // отпустить все блокировки
    }

\*) Т.о. идея противодействия дедлокам - если хоть одна блокировка занята, отпустить все и пытаться захватить заново. Идея весьма очевидная, но рабочая.  
Реализация же как обычно намного сложнее, чем кажется.

## <a id='toc7_3_'></a>["Контекст"](#toc0_)

Wait-Die контекст состоит из:

    списка захваченных блокировок;
    уникального "timestamp"

В качестве timestamp может быть что угодно уникальное для потока - например pid. Крайне желательно, чтобы timestamp монотонно возрастал.  

     void wdlock_ctx_init(struct wdlock_ctx *ctx) {
         static atomic_uulong timestamp;
         ctx->timestamp = atomic_fetch_add(&timestamp, 1) + 1;  // например ++атомарная переменная
         ctx->next = nullptr;
     }
    
Переполнение timestamp тут даже если и будет достигнуто (через годы или века работы приложения), то с практической точки зрения на возможность дедлоков не повлияет (главное уникальность в конкретный момент времени). Может повлиять только на живость потока, что зависит от конкретной реализации логики потока, а не от данного алгоритма.

## <a id='toc7_4_'></a>[Магия timestamp](#toc0_)

timestamp позволяет избегать deadlock-ов

    1. храним в каждой успешно захваченной блокировке сохраняется timestamp из wdlock_ctx, который использовали при захвате блокировки;
    2. при попытке захватить блокировку (реализация int wdlock(...)) возможно несколько вариантов:
        
        если блокировка свободна, то пытаемся ее захватить- как обычно (и записать свой timestamp);
        если блокировка занята, то нужно сравнить наш timestamp с сохраненным в блокировке:
            
            если наш timestamp меньше, чем timestamp блокировки, то ждем как в обычной блокировке (наш контекст "старше");
            в противном случае не ждем, а возвращаем признак неудачи (умираем), для этого и нужен return int:
                при этом отпускаем все захваченные локи (если более старшим они понадобятся)
                *) умираем, значит wdlock не сидит в цикле, а выходит по ретурну (этот момент как раз препятствует дедлоку),
                   уходим на второй круг, получаем более высокий timestamp, и по новой.
                **) т.к. за блокировку могут соревноваться несколько контекстов, то полезно их упорядочить по timestamp
                    в очередь, чтобы они быстрей не толкаясь (перехватывая локи, которые по-любому придется отпустать) могли
                    бы ее получить

Корректность. Поток ждет на блокировке, если timestamp блокировки больше, чем timestamp потока

    deadlock соответствует циклу в Wait-For графе;
    при использовании Wait-Die timestamp-ы блокировокна любом пути в графе строго возрастают;
    следовательно, цикла в Wait-For графе быть неможет.


В Linux дополнительно используется расширение wait/die блокировки - wound/wait.  
В контекст добавляется статус "ранен" вместо завершения ожидания блокировки и ухода на второй круг за новым timestamp.  
Только раненная блокировка с более высоким timestamp снимается со стека и перезапускается.  
Что это дает? Статус "ранен" проверяется только в случае конкуренции за блокировку. Блокировки уходят на второй круг не сразу, уменьшается количество вызовов функций. Типа того?  
Плюс блокировка идентифицирует конкурирующую блокировку (от кого она получила ранение)?  

It is recommended to allocate the context itself on the stack. Чтобы он самоуничтожался с выходом из блокировки?  